In [1]:
from functools import partial
import mysql.connector

connection = mysql.connector.connect(
    host='localhost',
    user='megadados',
    password='', #sua senha
    database='musica', #Nome database que quer acessar
)


def run_db_query(connection, query, args=None):
    with connection.cursor() as cursor:
        print('Executando query:')
        cursor.execute(query, args)
        for result in cursor:
            print(result)


db = partial(run_db_query, connection)

In [2]:
#Visualizando as tabelas
db('SHOW TABLES')

Executando query:
('autor',)
('cd',)
('cd_categoria',)
('faixa',)
('gravadora',)
('musica',)
('musica_autor',)


In [3]:
#Visualizando o schema
db('DESCRIBE musica')

Executando query:
('Codigo_Musica', b'int', 'NO', 'PRI', None, '')
('Nome_Musica', b'varchar(60)', 'YES', '', None, '')
('Duracao', b'int', 'YES', '', None, '')


## Pipeline do comando Select ##
```
SELECT [DISTINCT] coluna(s)
FROM tabela(s)
WHERE condição
GROUP BY grupo de uma coluna
HAVING condição (aplicada após GROUP BY)
ORDER BY expressao (pode ser coluna, ASC OU DESC)
LIMIT nº max resultados
OFFSET onde quer começar o limit
```  


A ordem de execução do comando `SELECT` é aproximadamente como segue:

1. `FROM <source_tables>`: indica as tabelas que serão usadas nesta query e, conceitualmente, combina estas tabelas através de *produto cartesiano* em uma grande tabela. (Note o termo "*conceitualmente*" que usei: em termos de implementação da query este produto cartesiano raramente é construído.)

2. `WHERE <filter_expression>`: filtra linhas.

3. `GROUP BY <grouping_expressions>`: agrupa conjuntos de linhas.

4. `SELECT <select_heading>`: escolha de colunas e de agregados.

5. `HAVING <filter_expression>`: outra filtragem, esta aplicada apenas **depois** da agregação. Pode usar resultados do processo de agregação. Obriga o uso de `GROUP BY`.

6. `DISTINCT`: Elimina linhas duplicadas.

7. `ORDER BY`: ordena as linhas do resultado.

8. `OFFSET <count>`: Pula linhas do resultado. Requer LIMIT.

9. `LIMIT <count>`: Mantém apenas um número máximo de linhas.

Esta sequencia também serve como dica de como projetar uma query! 
- Comece identificando as tabelas que você deseja usar
- Monte o filtro de linhas, incluindo critérios de `JOIN`
- Agrupe
- Selecione colunas e aplique funções de agregação, conforme necessário
- Filtre com `HAVING`, agora que temos agregação
- O resto é mais fácil, aplique conforme requerido

## Comando Create

In [4]:
db('DROP TABLE IF EXISTS Usuario')
db('''CREATE TABLE Usuario(
    nome VARCHAR(45),
    usuario_id INT NOT NULL AUTO_INCREMENT,
    musica_favorita_id INT,
    PRIMARY KEY (usuario_id),
    CONSTRAINT fk_musica_favorita FOREIGN KEY (musica_favorita_id)
        REFERENCES musica (Codigo_musica)
  )''')

Executando query:
Executando query:


## Comando Insert

In [5]:
db('''INSERT INTO Usuario (nome, musica_favorita_id) VALUES 
   ("Joao", 9 ),
   ("Carla", 1),
   ("Tiago", 19),
   ("Tomas", 19),
   ("Tatiana", 19),
   ("Temer", 19)
   ''')

Executando query:


In [6]:
db('SELECT * FROM Usuario')

Executando query:
('Joao', 1, 9)
('Carla', 2, 1)
('Tiago', 3, 19)
('Tomas', 4, 19)
('Tatiana', 5, 19)
('Temer', 6, 19)


## Comando Update

In [7]:
db('UPDATE Usuario SET musica_favorita_id = 5 WHERE nome LIKE "T%"')

Executando query:


In [8]:
db('SELECT * FROM Usuario')

Executando query:
('Joao', 1, 9)
('Carla', 2, 1)
('Tiago', 3, 5)
('Tomas', 4, 5)
('Tatiana', 5, 5)
('Temer', 6, 5)


## Joins + Temporary Table
### Inner Join
#### Juntar tabelas se e somente se  os valores que quer comparar existirem em ambas
#### Exemplo: Tabela com nome da pessoa e nome da música favorita
#### OBS: dropar a tabela quando terminar de usar

In [10]:
db('''DROP TABLE IF EXISTS Usuario_FavSong''')
db('''CREATE TEMPORARY TABLE Usuario_FavSong SELECT nome, Nome_musica FROM Usuario 
    INNER JOIN musica ON Usuario.musica_favorita_id = musica.Codigo_Musica
    ''')

Executando query:
Executando query:


In [11]:
db("SELECT * FROM Usuario_FavSong")

Executando query:
('Joao', 'Há Tempos')
('Carla', 'Será')
('Tiago', 'Tempo Perdido')
('Tomas', 'Tempo Perdido')
('Tatiana', 'Tempo Perdido')
('Temer', 'Tempo Perdido')


### Outer Left e Right Joins

In [12]:
db('''DROP TABLE IF EXISTS musicas_nofav ''')
db('''CREATE TEMPORARY TABLE  musicas_nofav SELECT Nome_Musica FROM musica LEFT OUTER JOIN Usuario 
    ON Usuario.musica_favorita_id = musica.Codigo_Musica
    WHERE Usuario.usuario_id IS NULL
    ''')

Executando query:
Executando query:


In [14]:
db("SELECT * FROM musicas_nofav LIMIT 2")

Executando query:
('Ainda é Cedo',)
('Geração Coca-Cola',)


### GROUP BY

In [16]:
### Selecionando duração total de trilhas por autor
db('''SELECT sum(Duracao), nome_autor FROM musica 
INNER JOIN musica_autor USING(codigo_musica) 
INNER JOIN autor USING(Codigo_autor) 
GROUP BY nome_autor LIMIT 5''')

Executando query:
(Decimal('61'), 'Renato Russo')
(Decimal('32'), 'Tom Jobim')
(Decimal('38'), 'Chico Buarque')
(Decimal('38'), 'Dado Villa-Lobos')
(Decimal('38'), 'Marcelo Bonfá')


In [51]:
### Agrupa os nomes de usuários se tiverem as mesmas músicas favoritas
db('''SELECT GROUP_CONCAT(nome SEPARATOR ", ") FROM Usuario GROUP BY musica_favorita_id 
''')

Executando query:
('Carla',)
('Tiago, Tomas, Tatiana, Temer',)
('Joao',)


## Modelo Relacional

![](ModeloRelacional.png)

## Operador LIKE

![](LikeOperator.png)